In [41]:
import requests
import xml.etree.ElementTree as ET
from tkinter import *
from tkinter import ttk

def get_usd_rate_by_date(date_str):
    url = "https://api.cba.am/exchangerates.asmx"
    headers = {
        "Content-Type": "text/xml; charset=utf-8",
        "SOAPAction": "http://www.cba.am/ExchangeRatesByDateByISO"
    }

    # Construct the SOAP XML body
    body = f"""<?xml version="1.0" encoding="utf-8"?>
    <soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
                   xmlns:xsd="http://www.w3.org/2001/XMLSchema"
                   xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
      <soap:Body>
        <ExchangeRatesByDateByISO xmlns="http://www.cba.am/">
          <date>{date_str}T00:00:00</date>
          <ISO>USD</ISO>
        </ExchangeRatesByDateByISO>
      </soap:Body>
    </soap:Envelope>"""

    # Send the SOAP request
    response = requests.post(url, headers=headers, data=body)

    if response.status_code != 200:
        print(f"HTTP error: {response.status_code}")
        print(response.text)
        return

    # Parse the XML response
    namespaces = {
        'soap': 'http://schemas.xmlsoap.org/soap/envelope/',
        'cba': 'http://www.cba.am/'
    }

    #print(namespaces)

    root = ET.fromstring(response.content)

    # Find the exchange rate info
    rate_elem = root.find('.//cba:ExchangeRate', namespaces)
    if rate_elem is None:
        print("USD exchange rate not found.")
        return
    
    #print(root)

    iso = rate_elem.find('cba:ISO', namespaces).text
    amount = rate_elem.find('cba:Amount', namespaces).text
    rate = rate_elem.find('cba:Rate', namespaces).text

    # print(f"USD Exchange Rate on {date_str}:")
    # print(f"  Rate    : {rate}")
    # print(f"  Amount  : {amount}")
    return rate, amount

def display_text():
    global entry
    date= entry.get()
    
    return date

def button_clicked():
    rate, amount = get_usd_rate_by_date(display_text())
    label.config(text=f"USD Exchange Rate on {rate}:")
    #label.config(text=f"  Amount  : {amount}")

   
root = Tk()

root.geometry("750x250")

label = Label(root, text = "Exchange Currency")

label.pack()

ttk.Button(root, text="GET", command= button_clicked).pack()





entry= Entry(root, width= 40)
entry.focus_set()
entry.pack()
#print(str(date_str))

    


#ttk.Button(root, text = "Input currency", width= 20, command= input_text)


# get_button = Button(root, 
#                    text="Get USD Rate", 
#                    command=button_clicked,
#                    activebackground="blue", 
#                    activeforeground="white",
#                    anchor="center",
#                    bd=3,
#                    bg="lightgray",
#                    cursor="hand2",
#                    disabledforeground="gray",
#                    fg="black",
#                    font=("Arial", 12),
#                    height=2,
#                    highlightbackground="black",
#                    highlightcolor="green",
#                    highlightthickness=2,
#                    justify="center",
#                    overrelief="raised",
#                    padx=10,
#                    pady=5,
#                    width=15,
#                    wraplength=100)

# get_button.pack(padx=20, pady=20)

root.mainloop()


#date_str = input("YYYY-MM-DD")

